In [31]:
import pandas as pd
import sqlite3

In [32]:
connection = sqlite3.connect("../data/checking-logs.sqlite")

In [33]:
query = """
WITH test_before AS (
    SELECT
        'before' AS time,
        AVG((deadlines - strftime('%s', DATETIME(first_commit_ts))) / 3600) AS avg_diff
    FROM 
        test AS T 
    INNER JOIN 
        deadlines AS D 
    ON 
        T.labname = D.labs
    WHERE 
        labname != 'project1' 
        AND first_commit_ts < first_view_ts
        AND EXISTS (
            SELECT 1 
            FROM test AS T2 
            WHERE T2.uid = T.uid 
            AND T2.first_commit_ts >= T2.first_view_ts
        )
),
test_after AS (
    SELECT
        'after' AS time,
        AVG((deadlines - strftime('%s', DATETIME(first_commit_ts))) / 3600) AS avg_diff
    FROM 
        test AS T 
    INNER JOIN 
        deadlines AS D 
    ON 
        T.labname = D.labs
    WHERE 
        labname != 'project1' 
        AND first_commit_ts >= first_view_ts
        AND EXISTS (
            SELECT 1 
            FROM test AS T2 
            WHERE T2.uid = T.uid 
            AND T2.first_commit_ts < T2.first_view_ts
        )
)
SELECT * FROM test_before
UNION
SELECT * FROM test_after;
"""
test_results = pd.io.sql.read_sql(query, connection)

In [34]:
query = """
WITH avg_first_view_ts AS (
    SELECT AVG(first_view_ts) AS avg_first_view_ts
    FROM test
),
control_before AS (
    SELECT
        'before' AS time,
        AVG((deadlines - strftime('%s', DATETIME(first_commit_ts))) / 3600) AS avg_diff
    FROM 
        control AS C 
    INNER JOIN 
        deadlines AS D ON C.labname = D.labs
    WHERE 
        C.labname != 'project1' 
        AND C.first_commit_ts < first_view_ts
        AND EXISTS (
            SELECT 1 
            FROM control AS C2 
            WHERE C2.uid = C.uid 
            AND C2.first_commit_ts >= first_view_ts
        )
),
control_after AS (
    SELECT
        'after' AS time,
        AVG((deadlines - strftime('%s', DATETIME(first_commit_ts))) / 3600) AS avg_diff
    FROM 
        control AS C 
    INNER JOIN 
        deadlines AS D ON C.labname = D.labs
    WHERE 
        C.labname != 'project1' 
        AND C.first_commit_ts >= first_view_ts
        AND EXISTS (
            SELECT 1 
            FROM control AS C2 
            WHERE C2.uid = C.uid 
            AND C2.first_commit_ts < first_view_ts
        )
)
SELECT * FROM control_before
UNION ALL
SELECT * FROM control_after;
"""
control_results = pd.io.sql.read_sql(query, connection)

In [35]:
connection.close()

In [36]:
difference_control = (control_results.loc[control_results["time"] == "after"]["avg_diff"].values[0]
                      -
                      control_results.loc[control_results["time"] == "before"]["avg_diff"].values[0])

difference_test = (test_results.loc[test_results["time"] == "after"]["avg_diff"].values[0]
                      -
                      test_results.loc[test_results["time"] == "before"]["avg_diff"].values[0])


print(f'Difference: {round(difference_test - difference_control, 2)}\nThat means the hypothesis turned out to be true!!')

Difference: 15.12
That means the hypothesis turned out to be true!!
